# STA 141B Assignment 2

Due __Jan 29, 2019__ by 11:59pm. Submit by editing this file, committing the changes with git, and then pushing to your private GitHub repo for the assignment. This assignment will be graded for correctness.

Please do not rename this file or delete the exercise cells, because it will interfere with our grading tools. Put your answers in new cells after each exercise. You can make as many new cells as you like. Use code cells for code and Markdown cells for text. Answer all questions with complete sentences.

The purpose of this assignment is to practice using data frames to index, slice, reshape, aggregate, and group data.

## Indexing

__Exercise 1.1 (10 points).__ Give three examples of indexing a data frame with `[ ]`, `.loc[ ]`, and `.iloc[ ]`, respectively. Explain how each of these indexing methods is different.

In [3]:
import pandas as pd

df = pd.DataFrame({"x": [1, 2, 3], "y": ["a", "b", "c"]})
df

,x,y
0,1,a
1,2,b
2,3,c


In [8]:
df['x'] # returns column named 'x'

0    1
1    2
2    3
Name: x, dtype: int64

In [17]:
df.loc[0] # returns row with index 0

x    1
y    a
Name: 0, dtype: object

In [21]:
df.iloc[2,1] # returns element in position (2,1)

'c'

* `[ ]` is by position, name, or condition (for integer indecies, only by name or condition)
* `.loc[ ]` is by position
* `.iloc[ ]` is by name or condition

__Exercise 1.2 (10 points).__ What do negative indexes (as in `x[-1]`) do in Python? Create and try three examples of negative indexes for lists, NumPy arrays, and data frames, respectively. Then explain what you think negative indexes do. Confirm your explanation by linking to a relevant page in the Python, NumPy, or Pandas documentation.

In [168]:
x = [1, 2, 3, 4, 5]
x[-1]

5

In [174]:
import numpy as np
y = np.array(['a', 'b', 'c', 'd', 'e'])
y[-1]

'e'

In [172]:
z = pd.DataFrame({"x": x, "y": y})
z.iloc[-1]

x    5
y    e
Name: 4, dtype: object

A negative index (like `x[-i]`) accesses the `i-1`-th from the last element in the container. In other words, if `len(x)=n` for some container `x`, then `x[-i]` would return `x[n-i]`.

See https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.indexing.html for documentation on NumPy array indexing.

__Exercise 1.3 (10 points).__ Give an example and explain Pandas' data alignment (or index alignment) feature.

In [121]:
x = pd.Series(["a", "b", "c"], index = [0, 1, 2])
y = pd.Series(["b", "a", "d", "c"], index = [0, 1, 2, 3])

x

0    a
1    b
2    c
dtype: object

In [122]:
y

0    b
1    a
2    d
3    c
dtype: object

In [123]:
x + y

0     ab
1     ba
2     cd
3    NaN
dtype: object

__Exercise 1.4 (10 points).__ Explain the difference between the similarly-named data frame methods `.reindex()` and `.reset_index()`. Give two examples to show what each method respectively does.

How might these methods be useful when combined with Pandas' data alignment feature?

*Hint: Besides the Pandas documentation, `.reindex()` is explained in Python for Data Analysis 5.2, and `.reset_index()` is explained [here](https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html#Index-setting-and-resetting).*

`.reindex()` assigns new indices specified in the arguments, whereas `.reset_index()` removes any assigned indices.

## Aquaculture Data Set 🐟🦐

The U.S. Department of Agriculture publishes data sets about U.S. food production online. In the next few exercises, you'll use the [U.S. Aquaculture data set](https://www.ers.usda.gov/webdocs/DataFiles/47759/AquacultureTradeFull.xls?v=661.2). The data set contains information about fish and shellfish imports and exports (more info [here](https://www.ers.usda.gov/data-products/aquaculture-data/)). The data set is published as a single Excel file.

__Exercise 1.5 (20 points).__ Download the aquaculture data set, open it in your favorite spreadsheet program, and take a moment to familiarize yourself with the data.

Use Pandas to _read_ the sheet that contains total imports from the Excel file. Clean the data frame so that it's _tidy_ (more info [here](http://shzhangji.com/blog/2017/09/30/pandas-and-tidy-data/) and [here](https://www.jstatsoft.org/article/view/v059i10)). In particular, make sure that:

* years are in a single column
* measurements are in a single column
* all observations indicate whether they are value or volume
* fish type and preparation are in separate columns
* footnotes are removed

For example, after tidying the data, I ended up with a multiindex series that looked like:

```
measurement     product  format             year
volume_1000lbs  Trout     fresh and frozen  1989    1467.892443
                                            1990    5898.067124
                                            1991    3866.117733
                                            1992    6200.029791
                                            1993    4744.615508
                                            ...
dtype: float64
```

Your result does not have to be identical, but should have a similar shape. Your result can be a data frame with multiple columns, or a multiindex series.

Convert your code into a function `read_aqua_mx()` with appropriate documentation and parameters. Test that your function can also correctly read the total exports sheet.

In [9]:
import pandas as pd
import numpy as np

In [157]:
aqua = pd.read_excel("AquacultureTradeFull.xls", sheet_name = 1, skiprows = 1)

aqua = aqua.drop([30, 31, 32])

for i in range(14):
    aqua.iloc[i, 0] = "Volume_1000lbs"

for i in range(14,30):
    aqua.iloc[i, 0] = "Value_$1000US"

aqua = aqua.rename(index = str, columns = {'Product': 'Measurement', 'Unnamed: 1': 'Product'})

aqua = aqua.set_index(['Measurement', 'Product'])
aqua = aqua.stack()

aqua = aqua.reset_index()

aqua['Product'] = aqua['Product'].str.rstrip('12345/ ')
aqua[['Product', 'Format']] = aqua['Product'].str.split(', ', expand=True)

aqua = aqua.rename(index = str, columns = {'level_2': 'Year', 0: 'Amount'})

aqua = aqua.reindex(columns=['Measurement', 'Product', 'Format', 'Year', 'Amount'])

aqua.head()

,Measurement,Product,Format,Year,Amount
0,Volume_1000lbs,Trout,fresh and frozen,1989,1467.892443
1,Volume_1000lbs,Trout,fresh and frozen,1990,5898.067124
2,Volume_1000lbs,Trout,fresh and frozen,1991,3866.117733
3,Volume_1000lbs,Trout,fresh and frozen,1992,6200.029791
4,Volume_1000lbs,Trout,fresh and frozen,1993,4744.615508


__Exercise 1.6 (20 points).__ What kind of fish or shellfish did the U.S. import the most of in 2017, and how much was imported? What was the total value of the most imported fish or shellfish, and was its value highest across all imports in 2017?

Was the most imported fish or shellfish different in previous years?

__Exercise 1.7 (20 points).__ Repeat the analysis from Exercise 1.6 for exports.

Are there any fish or shellfish that the U.S. imports and also exports?

__Exercise 1.8 (20 points).__ What are the top five countries the U.S. imported shrimp from in 2017? _You'll need to read another sheet from the data set for this question._

Use a chart to show how volume imported has changed over the years for all countries that have ever been a top five source of shrimp.